# Baseline RAG (retrieve → generate) and timeframe drift failure

This notebook loads the persisted Chroma index from Notebook 02 and runs a simple retrieve-then-generate baseline with OpenAI chat completions. It intentionally does **not** enforce recency so we can observe timeframe drift.

In [6]:
from __future__ import annotations

import os
import sys
from pathlib import Path

import pandas as pd


def _find_project_root() -> Path:
    cwd = Path.cwd().resolve()
    for base in (cwd, *cwd.parents):
        if (base / "src" / "config.py").exists():
            return base
        nested = base / "agentic-rag-second-brain"
        if (nested / "src" / "config.py").exists():
            return nested
    raise RuntimeError("Could not locate project root containing src/config.py")

PROJECT_ROOT = _find_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
os.chdir(PROJECT_ROOT)

from src.config import settings
from src.rag_baseline import baseline_rag_answer
from src.retrieval import load_persisted_index, retrieve_chunks

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "").strip()
OPENAI_MODEL = os.getenv("OPENAI_MODEL", settings.openai_model)
EMBED_MODEL = os.getenv("EMBED_MODEL", settings.embed_model)
CHROMA_DIR = Path(os.getenv("CHROMA_DIR", settings.chroma_dir)).resolve()
TOP_K = int(os.getenv("TOP_K", settings.top_k))
TEMPERATURE = float(os.getenv("TEMPERATURE", settings.temperature))
MAX_CONTEXT_CHARS = int(os.getenv("MAX_CONTEXT_CHARS", settings.max_context_chars))

print("Config:")
print(f"- PROJECT_ROOT: {PROJECT_ROOT}")
print(f"- CHROMA_DIR: {CHROMA_DIR}")
print(f"- EMBED_MODEL: {EMBED_MODEL}")
print(f"- OPENAI_MODEL: {OPENAI_MODEL}")
print(f"- TOP_K: {TOP_K}")
print(f"- TEMPERATURE: {TEMPERATURE}")
print(f"- MAX_CONTEXT_CHARS: {MAX_CONTEXT_CHARS}")
print(f"- OPENAI_API_KEY set: {'yes' if OPENAI_API_KEY else 'no'}")


Config:
- PROJECT_ROOT: C:\Repos\Intro-to-RAG-Agentic-RAG-2602\agentic-rag-second-brain
- CHROMA_DIR: C:\Repos\Intro-to-RAG-Agentic-RAG-2602\agentic-rag-second-brain\data\processed\chroma
- EMBED_MODEL: text-embedding-3-small
- OPENAI_MODEL: gpt-4o-mini
- TOP_K: 6
- TEMPERATURE: 0.0
- MAX_CONTEXT_CHARS: 10000
- OPENAI_API_KEY set: yes


In [7]:
if not OPENAI_API_KEY:
    raise EnvironmentError(
        "OPENAI_API_KEY is required for Notebook 03. "
        "Set it before running, for example: `export OPENAI_API_KEY='your-key'`."
    )


In [8]:
index = None
try:
    index = load_persisted_index(chroma_dir=CHROMA_DIR, embed_model=EMBED_MODEL)
    print(f"Loaded persisted index from: {CHROMA_DIR}")
except FileNotFoundError as err:
    print(str(err))
    print("Please run notebooks/02_indexing_chroma_llamaindex.ipynb first, then re-run this notebook.")


Loaded persisted index from: C:\Repos\Intro-to-RAG-Agentic-RAG-2602\agentic-rag-second-brain\data\processed\chroma


In [9]:
demo_queries = {
    "Q1 easy win": "What chunking overlap is currently recommended?",
    "Q2 drift question": "What embedding model should we use?",
}

demo_queries


{'Q1 easy win': 'What chunking overlap is currently recommended?',
 'Q2 drift question': 'What embedding model should we use?'}

In [10]:
if index is None:
    print("Skipping retrieval + generation because persisted index is unavailable.")
else:
    for label, query in demo_queries.items():
        print("\n" + "=" * 90)
        print(f"{label}: {query}")

        retrieved = retrieve_chunks(index=index, query=query, top_k=TOP_K)
        retrieved_df = pd.DataFrame(
            [
                {
                    "score": item["score"],
                    "doc_date": item["doc_date"],
                    "doc_title": item["doc_title"],
                    "chunk_id": item["chunk_id"],
                    "snippet": item["text"][:220].replace("\n", " "),
                }
                for item in retrieved
            ]
        )

        print("\nRetrieved chunks:")
        display(retrieved_df)

        result = baseline_rag_answer(
            index=index,
            query=query,
            top_k=TOP_K,
            model=OPENAI_MODEL,
            temperature=TEMPERATURE,
            max_context_chars=MAX_CONTEXT_CHARS,
        )

        print("\nAnswer:")
        print(result["answer"])
        if result.get("notes"):
            print("\nNotes:")
            print(result["notes"])

        citations_df = pd.DataFrame(result["citations"])
        print("\nCitations:")
        display(citations_df)



Q1 easy win: What chunking overlap is currently recommended?

Retrieved chunks:


,score,doc_date,doc_title,chunk_id,snippet
0,0.553763,2025-09-03,Chunking Strategy v2: Smaller Chunks + Overlap,20f6f8a9b43928b36d1bb1bb1d53b6391bf5eec0:9,New experiments show better grounding with sma...
1,0.433912,2025-11-12,Chunking Maintenance Checklist,2e91dc62f195312576077c02418c592e29888039:12,"Operationally, smaller chunks increased node c..."
2,0.431537,2025-04-14,Chunking Feedback from Pilot,86cf447369964142c025dd77b9b4b8b494698d90:4,Large chunk windows often blend unrelated sect...
3,0.413852,2025-02-02,Chunking Strategy v1: Large Windows,cbf9caa8525adeee4d0ca3138f6847bbe9360c10:1,Initial ingestion uses large chunks (1200 char...
4,0.335961,2025-10-02,Demo Retro: Internal Stakeholder Session,23d249bcab98910ef4133cf9ece7801e2b179a31:10,Stakeholders responded positively to timeline-...
5,0.272976,2025-05-22,Research Snippet: Hybrid Retrieval,79c9b8fbd7b9030b5f342d8f45587b0bb8c4a3fb:5,A short literature scan suggests dense+sparse ...



Answer:
The currently recommended chunking overlap is 60 characters, used in conjunction with smaller chunks of 420 characters. This approach helps maintain cleaner topical boundaries while providing necessary context bridges.

Citations:


,doc_title,doc_date,chunk_id,source_path
0,Chunking Strategy v2: Smaller Chunks + Overlap,2025-09-03,20f6f8a9b43928b36d1bb1bb1d53b6391bf5eec0:9,C:\Repos\Intro-to-RAG-Agentic-RAG-2602\agentic...



Q2 drift question: What embedding model should we use?

Retrieved chunks:


,score,doc_date,doc_title,chunk_id,snippet
0,0.427510,2025-01-10,Embedding Model Decision: Cost-First Default,627528ea8b8af5f59df5fae9b902a22869e8e53f:0,We should standardize on EmbedLite-v1 for now ...
1,0.407687,2025-07-05,Embedding Model Decision Update: Quality Priority,1843528f9966ef38e563dc60ec056795eab0a0b1:7,Query logs now show many semantically subtle q...
2,0.384486,2025-03-18,Q1 Embedding Evaluation Notes,aa23b701925fb16b6312fa7dc6f53474541c46fc:3,Compared EmbedLite-v1 and EmbedPro-v2 on histo...
3,0.343820,2025-10-21,Embedding Rollout Postmortem,3ce7ccdc6cae9be14a952f15d541a4f87c73ec51:11,"After switching to EmbedPro-v2, retrieval qual..."
4,0.263922,2025-05-22,Research Snippet: Hybrid Retrieval,79c9b8fbd7b9030b5f342d8f45587b0bb8c4a3fb:5,A short literature scan suggests dense+sparse ...
5,0.249269,2025-10-02,Demo Retro: Internal Stakeholder Session,23d249bcab98910ef4133cf9ece7801e2b179a31:10,Stakeholders responded positively to timeline-...



Answer:
The recommended embedding model to use is EmbedPro-v2, as it has been shown to improve retrieval quality for nuanced queries, despite a significant cost increase compared to EmbedLite-v1. This change was made due to the need for better handling of semantically subtle questions.

Notes:
The context indicates a transition from EmbedLite-v1 to EmbedPro-v2 due to quality concerns, but it does not provide information on any potential future changes or evaluations of the models.

Citations:


,doc_title,doc_date,chunk_id,source_path
0,Embedding Model Decision Update: Quality Priority,2025-07-05,1843528f9966ef38e563dc60ec056795eab0a0b1:7,C:\Repos\Intro-to-RAG-Agentic-RAG-2602\agentic...
1,Embedding Rollout Postmortem,2025-10-21,3ce7ccdc6cae9be14a952f15d541a4f87c73ec51:11,C:\Repos\Intro-to-RAG-Agentic-RAG-2602\agentic...


### Why baseline RAG can fail under timeframe drift

This baseline pipeline retrieves semantically similar chunks and asks the model to answer from that context only. Because retrieval can surface chunks from different dates (older and newer recommendations), the model may blend or choose outdated guidance. In the next (agentic) notebook, we will add explicit temporal reasoning and conflict handling to improve consistency.